In [1]:
import pandas as pd
from datetime import date
import time
pd.set_option('display.max_columns', None) # PAra que se vean todas las columnas de los df

In [2]:
df = pd.read_csv(r'Data/scopus_bd_aumentada.csv', sep=",")

C:\Users\YORLEY\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7,21,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [44]:
df_final = df[['Author(s)_ID_Rev','Title','Year','Source_title','Document_Type','Abstract' ]][df['Document_Type']=='Article']
df_final.dropna(subset=['Author(s)_ID_Rev','Title'],inplace=True)
df_final.dropna(subset=['Author(s)_ID_Rev'],inplace=True)

In [45]:
df_authors = df[['Authors_Rev','Author(s)_ID_Rev']][:].drop_duplicates(subset='Author(s)_ID_Rev')

### Total Citaciones

In [46]:
df_temp=df[['Author(s)_ID_Rev', 'Cited_by']].groupby(by='Author(s)_ID_Rev').sum()
df_temp.reset_index(level=0, inplace=True)

In [47]:
df_authors = df_authors.merge(right=df_temp, how='inner', on='Author(s)_ID_Rev')

### Total Artículos publicados

In [48]:
df_temp=df[['Author(s)_ID_Rev', 'Title']].groupby(by='Author(s)_ID_Rev').count()
#Convierte el indice en otro campo
df_temp.reset_index(level=0, inplace=True)

In [49]:
df_authors = df_authors.merge(df_temp, how='inner', on ='Author(s)_ID_Rev')

### Máximo número de citaciones

In [50]:
df_temp=df[['Author(s)_ID_Rev', 'Cited_by']].groupby(by='Author(s)_ID_Rev').max()
df_temp.reset_index(level=0, inplace=True)

In [51]:
df_authors = df_authors.merge(right=df_temp, how='inner', on='Author(s)_ID_Rev')
df_authors.columns = ['Authors_Rev', 'Author(s)_ID_Rev', 'Total_Citation', 'Total_Title', 'Max_Citation']
df_authors['Max_Citation'].fillna(0, inplace=True)

### Número de Revistas

In [52]:
df_temp=df[['Author(s)_ID_Rev', 'Source_title']].groupby(by=['Author(s)_ID_Rev']).nunique()
df_temp = df_temp[['Source_title']]
df_temp.reset_index(level=0, inplace=True)
df_temp.columns = ['Author(s)_ID_Rev','Total_Journal']
df_temp.sort_values(by='Total_Journal', ascending = False, inplace= True)

In [53]:
df_authors = df_authors.merge(df_temp, how='inner', on ='Author(s)_ID_Rev')

In [54]:
año_actual=int(date.today().strftime("%Y"))
año_pasado=año_actual-1

In [55]:
df2_años=df[(df['Year']==año_actual) | (df['Year']==año_pasado) ]

### Total Publicaciones últimos dos años

In [56]:
df_temp=df2_años[['Author(s)_ID_Rev', 'Title']].groupby(by='Author(s)_ID_Rev').count()
df_temp.reset_index(level=0, inplace=True)
df_temp.columns = ['Author(s)_ID_Rev','Delta_publications']
df_authors = df_authors.merge(df_temp, how='left', on ='Author(s)_ID_Rev')

In [57]:
df_authors['Delta_publications'].fillna(0, inplace=True)


### Carrera Autor

In [58]:
df_temp = df[['Author(s)_ID_Rev','Year']].groupby(by = 'Author(s)_ID_Rev').min()
df_temp.reset_index(level=0, inplace=True)
df2_temp = df[['Author(s)_ID_Rev','Year']].groupby(by = 'Author(s)_ID_Rev').max()
df2_temp.reset_index(level=0, inplace=True)
df_temp = df_temp.merge(df2_temp, how='left', on='Author(s)_ID_Rev')
df_temp['autor_career'] =  df_temp['Year_y'] - df_temp['Year_x']
df_authors = df_authors.merge(df_temp[['Author(s)_ID_Rev','autor_career']], how='left', on ='Author(s)_ID_Rev')

### Citación Media por Revista

In [59]:
df_temp = df[['Source_title','Cited_by']].groupby(by='Source_title').mean().fillna(0)
df_temp.reset_index(level=0, inplace=True)

In [60]:
df2_temp = df[['Author(s)_ID_Rev','Source_title','Cited_by']].groupby(by=['Author(s)_ID_Rev','Source_title']).mean().fillna(0)
df2_temp.reset_index(level=['Author(s)_ID_Rev','Source_title'],inplace=True)
df2_temp =df2_temp.merge(df_temp, how='left', on='Source_title')
df2_temp = df2_temp[['Author(s)_ID_Rev','Cited_by_y']].groupby(by='Author(s)_ID_Rev').mean()
df2_temp.reset_index(level=0,inplace=True)
df2_temp.columns = ['Author(s)_ID_Rev','citation_journal']
df_authors = df_authors.merge(df2_temp, how='left', on='Author(s)_ID_Rev')

### Total Coautores ultimos dos años

#### Leer
* Identificación de Todos los autores
* para cada autor se revisan los papers que en los que aparece
* para cada paper de ese autor se revisan cuantas veces aparece bajo un nombre diferente al del autor superior
* si el autor que aparece no se ha añadido a la lista de coautores se hace.

# Esto se demora corriendo alrededor de 10 minutos
## puedes irte a tomar una bebida caliente mientras tanto
### es en serio :( :(

In [61]:
start = time.time()
#Línea de prueba para correr rápido
#lista_autores = df2_años['Author(s)_ID_Rev'][df2_años['Author(s)_ID_Rev']=='53866250100'].unique()
lista_autores = df2_años['Author(s)_ID_Rev'].unique()[:]
lista_coautores = []
for autor in lista_autores:
    df_paper = df2_años[['Author(s)_ID_Rev','Title']][df2_años['Author(s)_ID_Rev']==autor]
    lista_papers = df_paper['Title']
    coautores = []
    for paper in lista_papers:
        coautor = (df2_años[['Author(s)_ID_Rev','Title']][(df2_años['Title']==paper)&(df2_años['Author(s)_ID_Rev']!=autor)]
                   ['Author(s)_ID_Rev'])
        if len(coautor)==0:
            continue
        else:
            coautor = (df2_años[['Author(s)_ID_Rev','Title']][(df2_años['Title']==paper)&(df2_años['Author(s)_ID_Rev']!=autor)]
                   ['Author(s)_ID_Rev'])
        for c in coautor:
            if c is not coautores:
                coautores.append(c)
    del(df_paper)
    lista_coautores.append([autor,len(coautores)])
end = time.time()
#print((end - start)/60)

In [62]:
#Tiempo de ejecución
(end -start)/60

4.9693898757298784

### lista_coautores

In [64]:
df2_temp = pd.DataFrame(lista_coautores, columns=['Author(s)_ID_Rev','Delta_coautores'])

In [67]:
df_authors = df_authors.merge(df2_temp, how='left', on='Author(s)_ID_Rev').sort_values(by='Delta_publications', ascending = False)

In [68]:
# Los autores que no tuvieron publicaciones en los últimos dos años, reportan Nan en coautores
df_authors['Delta_coautores'].fillna(0, inplace=True)

In [70]:
#Aun hay datos nulos
df_authors.isnull().any()

Authors_Rev           False
Author(s)_ID_Rev      False
Total_Citation        False
Total_Title           False
Max_Citation          False
Total_Journal         False
Delta_publications    False
autor_career          False
citation_journal      False
Delta_coautores       False
dtype: bool

# Indicadores Papers

* Citaciones por Artículos
* años del artículos
* Cantidad de coautures

In [81]:
df_papers =df[['Title','Cited_by']].drop_duplicates(subset='Title').fillna(0)
df_temp = df[['Title','Year']].drop_duplicates(subset='Title').fillna(0)
df_temp['Year'] = año_actual - df_temp['Year']
df_papers = df_papers.merge(df_temp, how='left', on = 'Title')
df_temp = df[['Title', 'Author(s)_ID_Rev']].fillna(0).groupby(by='Title').count()
df_temp.reset_index(level=0, inplace= True)
df_papers = df_papers.merge(df_temp, how='left', on = 'Title')
df_papers.columns = ['Title', 'Cited_by', 'Years', 'cant_coautores']

### Citaciones por revista

In [83]:
#Revistas y Citaciones
df_temp = df[['Source_title','Cited_by']].groupby(by='Source_title').mean().fillna(0)
df_temp.reset_index(level=0, inplace=True)

In [87]:
df2_temp = df[['Title','Source_title','Cited_by']].groupby(by=['Title','Source_title']).mean().fillna(0)
df2_temp.reset_index(level=['Title','Source_title'],inplace=True)
df2_temp =df2_temp.merge(df_temp, how='left', on='Source_title')
df2_temp = df2_temp[['Title','Cited_by_y']].groupby(by='Title').mean()
df2_temp.reset_index(level=0,inplace=True)
df2_temp.columns = ['Title','journal_mean_citation']
df_papers = df_papers.merge(df2_temp, how='left', on='Title')

In [89]:
#Hay datos nulos
df_papers.isna().any()

Title                    False
Cited_by                 False
Years                    False
cant_coautores           False
journal_mean_citation    False
dtype: bool

### Cosolidacion de los datos

In [90]:
# Creacion de un df vacio que se utilizará para consolidar toda la información
df_consol = (df_final
             .merge(df_authors, how='left', on='Author(s)_ID_Rev')
             .merge(df_papers, how='left', on='Title')
            )

In [94]:
#df_authors.to_csv('indicadores_autores.csv')
#df_papers.to_csv('indicadores_papers.csv')
df_consol.to_csv('Data/scopus_con_indicadores.csv')